# Cloud Noise Removal

A couple of sites have some "sky bananas" or cloud noise. As there's only two I will manually remove them.

The two sites are:
- ULM_325
- ULM_147

In [1]:
import pdal
import numpy as np
import pandas as pd

import pyvista as pv
pv.set_jupyter_backend("client")

from pathlib import Path

# Creating Backups

In [2]:
site_lidar_dir = Path("../data/outputs/sites/lidar")
backup_dir = site_lidar_dir.parent / "lidar-with-cloud-noise"

backup_dir.mkdir(parents=True, exist_ok=True)

In [3]:
import shutil

def create_backup(site):
    original_file = site_lidar_dir / f"{site}.copc.laz"    
    backup_file = backup_dir / f"{site}.copc.laz"

    shutil.copy2(original_file, backup_file)

create_backup("ULM_325")
create_backup("ULM_147")

In [4]:
def center_and_size_to_box(center, size):
    (cx, cy, cz) = center
    (sx, sy, sz) = size
    return (cx - 0.5 * sx, cx + 0.5 * sx,
            cy - 0.5 * sy, cy + 0.5 * sy,
            cz - 0.5 * sz, cz + 0.5 * sz)

## ULM 325

In [5]:
# The values of the box were manually determined by inspecting the point cloud
# in cloudcompare.
ulm_325_box_center = (476110.031, 5230827.372, 87.567)
ulm_325_box_size = (53.144, 49.606, 36.957)

ulm_325_clip_box = center_and_size_to_box(ulm_325_box_center, ulm_325_box_size)
(minx, maxx, miny, maxy, minz, maxz) = ulm_325_clip_box

assign_expressions = [
    f"Classification = 18 WHERE X >= {minx} && X <= {maxx} && Y >= {miny} && Y <= {maxy} && HeightAboveGround >= {minz} && HeightAboveGround <= {maxz}"
]

pipeline = ( pdal.Reader(str(backup_dir / "ULM_325.copc.laz")) 
    | pdal.Filter("filters.assign", value=assign_expressions) 
    | pdal.Writer(str(site_lidar_dir / "ULM_325.copc.laz"), type="writers.copc",  forward="scale,offset", extra_dims= "all")
)

pipeline.execute()

2500328

## ULM 147

I wasn't able to create a single bounding box to capture all the ULM 147 noise, instead I need two boxes.

In [6]:
# The values of the box were manually determined by inspecting the point cloud
# in cloudcompare.
ulm_147_boxA_center = (457919.822, 5285528.094, 121.494)
ulm_147_boxA_size = (95.724, 93.489, 82.994)

ulm_147_boxB_center = (457945.263, 5285589.062, 120.843)
ulm_147_boxB_size = (58.202, 22.717, 29.792)

ulm_147_clip_boxA = center_and_size_to_box(ulm_147_boxA_center, ulm_147_boxA_size)
(minxa, maxxa, minya, maxya, minza, maxza) = ulm_147_clip_boxA

ulm_147_clip_boxB = center_and_size_to_box(ulm_147_boxB_center, ulm_147_boxB_size)
(minxb, maxxb, minyb, maxyb, minzb, maxzb) = ulm_147_clip_boxB

assign_expressions = [
    f"Classification = 18 WHERE X >= {minxa} && X <= {maxxa} && Y >= {minya} && Y <= {maxya} && HeightAboveGround >= {minza} && HeightAboveGround <= {maxza}",
    f"Classification = 18 WHERE X >= {minxb} && X <= {maxxb} && Y >= {minyb} && Y <= {maxyb} && HeightAboveGround >= {minzb} && HeightAboveGround <= {maxzb}",
    # Because I missed one
    "Classification = 18 WHERE HeightAboveGround > 100"
]

pipeline = ( pdal.Reader(str(backup_dir / "ULM_147.copc.laz")) 
    | pdal.Filter("filters.assign", value=assign_expressions)
    | pdal.Writer(str(site_lidar_dir / "ULM_147.copc.laz"), type="writers.copc",  forward="scale,offset", extra_dims= "all")
)

pipeline.execute()

2076852

# Visualising the results

Points classified as cloud noise (using the Classification 18 for high noise) are shown in dark red below.

In [19]:
def read_point_cloud(file_path):
    pipeline = pdal.Reader(file_path).pipeline()
    pipeline.execute()
    return pipeline.arrays[0]

def plot_point_cloud(nd):
    X = nd["X"]
    Y = nd["Y"]
    Z = nd["HeightAboveGround"]
    positions = np.column_stack((X, Y, Z)).astype("float32")


    # Highlight any points with classification 18 (high noise)
    # also highlight classification 7 (low noise)
    classification = nd["Classification"]
    colors = np.zeros(positions.shape, dtype=np.uint8)
    colors[:] = [173, 216, 230]     # Pale blue 
    colors[classification == 18] = [139, 15, 30] # Dark red
    colors[classification == 7] = [255, 193, 37] # Goldenrod
    
    mesh = pv.PolyData(positions)
    mesh['colors'] = colors

    mesh.plot(point_size=4, scalars='colors', rgb=True)

In [20]:
ulm_325 = read_point_cloud(site_lidar_dir / "ULM_325.copc.laz")
ulm_147 = read_point_cloud(site_lidar_dir / "ULM_147.copc.laz")
plot_point_cloud(ulm_325)
plot_point_cloud(ulm_147)

Widget(value='<iframe src="http://localhost:50357/index.html?ui=P_0x16c19e710_2&reconnect=auto" class="pyvista…

Widget(value='<iframe src="http://localhost:50357/index.html?ui=P_0x176ed0f50_3&reconnect=auto" class="pyvista…